# Lab 6

You are tasked with evaluating card counting strategies for black jack. In order to do so, you will use object oriented programming to create a playable casino style black jack game where a computer dealer plays against $n$ computer players and possibily one human player. If you don't know the rules of blackjack or card counting, please google it. 

A few requirements:
* The game should utilize multiple 52-card decks. Typically the game is played with 6 decks.
* Players should have chips.
* Dealer's actions are predefined by rules of the game (typically hit on 16). 
* The players should be aware of all shown cards so that they can count cards.
* Each player could have a different strategy.
* The system should allow you to play large numbers of games, study the outcomes, and compare average winnings per hand rate for different strategies.

1. Begin by creating a classes to represent cards and decks. The deck should support more than one 52-card set. The deck should allow you to shuffle and draw cards. Include a "plastic" card, placed randomly in the deck. Later, when the plastic card is dealt, shuffle the cards before the next deal.

In [1]:
import random

class Card:
    def __init__(self, suit, rank):
        self.suit = suit
        self.rank = rank

    def __repr__(self):
        return f"{self.rank} of {self.suit}"

class Deck:
    def __init__(self, num_sets=1):
        self.suits = ['Hearts', 'Diamonds', 'Clubs', 'Spades']
        self.ranks = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'Jack', 'Queen', 'King', 'Ace']
        self.cards = []
        self.plastic_card = "Plastic Card"

        for _ in range(num_sets):
            for suit in self.suits:
                for rank in self.ranks:
                    self.cards.append(Card(suit, rank))
        #Add the plastic card to the deck
        self.cards.append(self.plastic_card)

    def shuffle(self):
        random.shuffle(self.cards)

    def draw(self):
        if not self.cards:
            print("The deck is empty.")

        drawn_card = self.cards.pop(0)

        if drawn_card == self.plastic_card:
            print("Plastic card drawn. Shuffling the deck.")
            self.shuffle()

        return drawn_card

    def __repr__(self):
        return f"Deck with {len(self.cards)} cards"


2. Now design your game on a UML diagram. You may want to create classes to represent, players, a hand, and/or the game. As you work through the lab, update your UML diagram. At the end of the lab, submit your diagram (as pdf file) along with your notebook. 

3. Begin with implementing the skeleton (ie define data members and methods/functions, but do not code the logic) of the classes in your UML diagram.

In [ ]:
from typing import List, Optional

class Card:
    def __init__(self, suit: str, rank: str):
        self.suit = suit
        self.rank = rank

    def value(self) -> int:
        pass

class Deck:
    def __init__(self, num_decks: int):
        self.cards = []
        self.num_decks = num_decks
        self.reset()

    def reset(self):
        pass

    def shuffle(self):
        pass

    def deal_card(self) -> Card:
        pass

class Player:
    def __init__(self, name: str, is_automated: bool, chips: int):
        self.name = name
        self.is_automated = is_automated
        self.chips = chips
        self.hand = []

    def place_bet(self) -> int:
        pass

    def decide_action(self, dealer_up_card: Card) -> str:
        pass

    def reset_hand(self):
        pass

class Dealer:
    def __init__(self):
        self.hand = []

    def decide_action(self) -> str:
        pass

    def reset_hand(self):
        pass

class BlackjackGame:
    def __init__(self, num_players: int, num_automated_players: int, num_decks: int):
        self.deck = Deck(num_decks)
        self.dealer = Dealer()
        self.players = self._create_players(num_players, num_automated_players)
        self.round_number = 0

    def _create_players(self, num_players: int, num_automated_players: int) -> List[Player]:
        pass

    def play_round(self):
        pass

    def reset_game(self):
        pass

class Simulation:
    def __init__(self, num_games: int, num_players: int, num_automated_players: int, num_decks: int):
        self.num_games = num_games
        self.num_players = num_players
        self.num_automated_players = num_automated_players
        self.num_decks = num_decks
        self.results = []

    def run_simulation(self):
        pass

    def analyze_results(self):
        pass

if __name__ == "__main__":
    # Create a simulation of 1000 games with 1 human player, 3 automated players, and 6 decks
    simulation = Simulation(num_games=1000, num_players=4, num_automated_players=3, num_decks=6)
    simulation.run_simulation()
    simulation.analyze_results()

4. Complete the implementation by coding the logic of all functions. For now, just implement the dealer player and human player.

In [8]:
import random
from typing import List, Optional

class Card:
    def __init__(self, suit: str, rank: str):
        self.suit = suit
        self.rank = rank

    def value(self) -> int:
        if self.rank in ['J', 'Q', 'K']:
            return 10
        elif self.rank == 'A':
            return 11
        else:
            return int(self.rank)

    def __repr__(self):
        return f"{self.rank} of {self.suit}"

class Deck:
    def __init__(self, num_decks: int):
        self.cards = []
        self.num_decks = num_decks
        self.reset()

    def reset(self):
        suits = ['Spade', 'Diamond', 'Club', 'Heart']
        ranks = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A']
        self.cards = [Card(suit, rank) for suit in suits for rank in ranks] * self.num_decks
        self.shuffle()

    def shuffle(self):
        random.shuffle(self.cards)

    def deal_card(self) -> Card:
        return self.cards.pop()

class Player:
    def __init__(self, name: str, is_automated: bool, chips: int):
        self.name = name
        self.is_automated = is_automated
        self.chips = chips
        self.hand = []
        self.bet = 0

    def place_bet(self) -> int:
        if self.is_automated:
            self.bet = random.randint(1, min(100, self.chips))
            self.chips -= self.bet
            return self.bet
        else:
            while True:
                try:
                    bet = int(input(f"{self.name}, you have {self.chips} chips. Place your bet: "))
                    if 0 < bet <= self.chips:
                        self.bet = bet
                        self.chips -= bet
                        return bet
                    else:
                        print("Invalid bet amount. Try again.")
                except ValueError:
                    print("Please enter a valid number.")

    def decide_action(self, dealer_up_card: Card) -> str:
        if self.is_automated:
            # Basic strategy: hit if hand value < 16, stand otherwise
            if self.hand_value() < 16:
                return 'hit'
            else:
                return 'stand'
        else:
            while True:
                action = input(f"{self.name}, do you want to hit or stand? ").strip().lower()
                if action in ['hit', 'stand']:
                    return action
                else:
                    print("Invalid action. Please type 'hit' or 'stand'.")

    def hand_value(self) -> int:
        value = sum(card.value() for card in self.hand)
        aces = sum(1 for card in self.hand if card.rank == 'A')
        while value > 21 and aces:
            value -= 10
            aces -= 1
        return value

    def reset_hand(self):
        self.hand = []
        self.bet = 0

    def __repr__(self):
        return f"{self.name} (Chips: {self.chips})"

class Dealer:
    def __init__(self):
        self.hand = []

    def decide_action(self) -> str:
        if self.hand_value() < 16:
            return 'hit'
        else:
            return 'stand'

    def hand_value(self) -> int:
        value = sum(card.value() for card in self.hand)
        aces = sum(1 for card in self.hand if card.rank == 'A')
        while value > 21 and aces:
            value -= 10
            aces -= 1
        return value

    def reset_hand(self):
        self.hand = []

    def __repr__(self):
        return "Dealer"

class BlackjackGame:
    def __init__(self, num_players: int, num_automated_players: int, num_decks: int):
        self.deck = Deck(num_decks)
        self.dealer = Dealer()
        self.players = self._create_players(num_players, num_automated_players)
        self.round_number = 0

    def _create_players(self, num_players: int, num_automated_players: int) -> List[Player]:
        players = []
        for i in range(num_players):
            if i < num_automated_players:
                players.append(Player(f"Player {i+1}", is_automated=True, chips=1000))
            else:
                players.append(Player("User", is_automated=False, chips=1000))
        return players

    def play_round(self):
        self.round_number += 1
        print(f"\n--- Round {self.round_number} ---")

        user = next((player for player in self.players if player.name == "User"), None)
        if user and user.chips <= 0:
            print(f"{user.name} has run out of chips. Game over!")
            return False

        for player in self.players:
            player.place_bet()

        for _ in range(2):
            for player in self.players:
                player.hand.append(self.deck.deal_card())
            self.dealer.hand.append(self.deck.deal_card())

        print(f"\nDealer's Hand: [{self.dealer.hand[0]}, ?]")
        for player in self.players:
            print(f"{player.name}'s Hand: {player.hand} (Value: {player.hand_value()})")

        for player in self.players:
            while player.hand_value() < 21:
                action = player.decide_action(self.dealer.hand[0])
                if action == 'hit':
                    player.hand.append(self.deck.deal_card())
                    print(f"{player.name} hits. Hand: {player.hand} (Value: {player.hand_value()})")
                else:
                    print(f"{player.name} stands.")
                    break

        print(f"\nDealer's Hand: {self.dealer.hand} (Value: {self.dealer.hand_value()})")
        while self.dealer.decide_action() == 'hit':
            self.dealer.hand.append(self.deck.deal_card())
            print(f"Dealer hits. Hand: {self.dealer.hand} (Value: {self.dealer.hand_value()})")

        dealer_value = self.dealer.hand_value()
        for player in self.players:
            player_value = player.hand_value()
            if player_value > 21:
                print(f"{player.name} busts! Loses {player.bet} chips.")
            elif dealer_value > 21 or player_value > dealer_value:
                player.chips += 2 * player.bet
                print(f"{player.name} wins! Wins {player.bet} chips.")
            elif player_value == dealer_value:
                player.chips += player.bet
                print(f"{player.name} pushes. Gets back {player.bet} chips.")
            else:
                print(f"{player.name} loses! Loses {player.bet} chips.")

        for player in self.players:
            player.reset_hand()
        self.dealer.reset_hand()

        return True

    def reset_game(self):
        self.deck.reset()
        for player in self.players:
            player.reset_hand()
        self.dealer.reset_hand()

class Simulation:
    def __init__(self, num_games: int, num_players: int, num_automated_players: int, num_decks: int):
        self.num_games = num_games
        self.num_players = num_players
        self.num_automated_players = num_automated_players
        self.num_decks = num_decks
        self.results = []

    def run_simulation(self):
        game = BlackjackGame(self.num_players, self.num_automated_players, self.num_decks)
        for _ in range(self.num_games):
            if not game.play_round():
                break
            self.results.append([player.chips for player in game.players])
            game.reset_game()

    def analyze_results(self):
        for i, player_results in enumerate(zip(*self.results)):
            avg_chips = sum(player_results) / len(player_results)
            print(f"Player {i+1} average chips per game: {avg_chips}")

if __name__ == "__main__":
    # Create a simulation of 1000 games with 1 human player, 3 automated players, and 6 decks
    simulation = Simulation(num_games=5, num_players=4, num_automated_players=3, num_decks=6)
    simulation.run_simulation()
    simulation.analyze_results()


--- Round 1 ---


User, you have 1000 chips. Place your bet:  1000



Dealer's Hand: [A of Heart, ?]
Player 1's Hand: [J of Heart, 10 of Club] (Value: 20)
Player 2's Hand: [7 of Spade, J of Club] (Value: 17)
Player 3's Hand: [2 of Spade, 9 of Heart] (Value: 11)
User's Hand: [6 of Diamond, 4 of Spade] (Value: 10)
Player 1 stands.
Player 2 stands.
Player 3 hits. Hand: [2 of Spade, 9 of Heart, 8 of Spade] (Value: 19)
Player 3 stands.


User, do you want to hit or stand?  stand


User stands.

Dealer's Hand: [A of Heart, 5 of Heart] (Value: 16)
Dealer hits. Hand: [A of Heart, 5 of Heart, 3 of Diamond] (Value: 19)
Player 1 wins! Wins 82 chips.
Player 2 loses! Loses 20 chips.
Player 3 pushes. Gets back 59 chips.
User loses! Loses 1000 chips.

--- Round 2 ---
User has run out of chips. Game over!
Player 1 average chips per game: 1082.0
Player 2 average chips per game: 980.0
Player 3 average chips per game: 1000.0
Player 4 average chips per game: 0.0


5.  Test. Demonstrate game play. For example, create a game of several dealer players and show that the game is functional through several rounds.

6. Implement a new player with the following strategy:

    * Assign each card a value: 
        * Cards 2 to 6 are +1 
        * Cards 7 to 9 are 0 
        * Cards 10 through Ace are -1
    * Compute the sum of the values for all cards seen so far.
    * Hit if sum is very negative, stay if sum is very positive. Select a threshold for hit/stay, e.g. 0 or -2.  

In [ ]:
import random
from typing import List, Optional

class Card:
    def __init__(self, suit: str, rank: str):
        self.suit = suit
        self.rank = rank

    def value(self) -> int:
        if self.rank in ['J', 'Q', 'K']:
            return 10
        elif self.rank == 'A':
            return 11
        else:
            return int(self.rank)

    def __repr__(self):
        return f"{self.rank} of {self.suit}"

class Deck:
    def __init__(self, num_decks: int):
        self.cards = []
        self.num_decks = num_decks
        self.reset()

    def reset(self):
        suits = ['Spade', 'Diamond', 'Club', 'Heart']
        ranks = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A']
        self.cards = [Card(suit, rank) for suit in suits for rank in ranks] * self.num_decks
        self.shuffle()

    def shuffle(self):
        random.shuffle(self.cards)

    def deal_card(self) -> Card:
        return self.cards.pop()

class Player:
    def __init__(self, name: str, is_automated: bool, chips: int):
        self.name = name
        self.is_automated = is_automated
        self.chips = chips
        self.hand = []
        self.bet = 0

    def place_bet(self) -> int:
        if self.is_automated:
            self.bet = random.randint(1, min(100, self.chips))
            self.chips -= self.bet
            return self.bet
        else:
            while True:
                try:
                    bet = int(input(f"{self.name}, you have {self.chips} chips. Place your bet: "))
                    if 0 < bet <= self.chips:
                        self.bet = bet
                        self.chips -= bet
                        return bet
                    else:
                        print("Invalid bet amount. Try again.")
                except ValueError:
                    print("Please enter a valid number.")

    def decide_action(self, dealer_up_card: Card) -> str:
        if self.is_automated:
            # Basic strategy: hit if hand value < 16, stand otherwise
            if self.hand_value() < 16:
                return 'hit'
            else:
                return 'stand'
        else:
            while True:
                action = input(f"{self.name}, do you want to hit or stand? ").strip().lower()
                if action in ['hit', 'stand']:
                    return action
                else:
                    print("Invalid action. Please type 'hit' or 'stand'.")

    def hand_value(self) -> int:
        value = sum(card.value() for card in self.hand)
        aces = sum(1 for card in self.hand if card.rank == 'A')
        while value > 21 and aces:
            value -= 10
            aces -= 1
        return value

    def reset_hand(self):
        self.hand = []
        self.bet = 0

    def __repr__(self):
        return f"{self.name} (Chips: {self.chips})"

class CardCountingPlayer(Player):
    def __init__(self, name: str, is_automated: bool, chips: int, threshold: int = 0):
        super().__init__(name, is_automated, chips)
        self.running_count = 0  # Tracks the running count of card values
        self.threshold = threshold  # Threshold for hit/stay decision

    def update_running_count(self, card: Card):
        if 2 <= card.value() <= 6:
            self.running_count += 1
        elif 7 <= card.value() <= 9:
            self.running_count += 0
        elif 10 <= card.value() <= 11:
            self.running_count -= 1

    def decide_action(self, dealer_up_card: Card) -> str:
        if self.running_count < self.threshold:
            return 'hit'
        else:
            return 'stand'

    def reset_hand(self):
        for card in self.hand:
            self.update_running_count(card)
        super().reset_hand()

class Dealer:
    def __init__(self):
        self.hand = []

    def decide_action(self) -> str:
        if self.hand_value() < 16:
            return 'hit'
        else:
            return 'stand'

    def hand_value(self) -> int:
        value = sum(card.value() for card in self.hand)
        aces = sum(1 for card in self.hand if card.rank == 'A')
        while value > 21 and aces:
            value -= 10
            aces -= 1
        return value

    def reset_hand(self):
        self.hand = []

    def __repr__(self):
        return "Dealer"

class BlackjackGame:
    def __init__(self, num_players: int, num_automated_players: int, num_decks: int):
        self.deck = Deck(num_decks)
        self.dealer = Dealer()
        self.players = self._create_players(num_players, num_automated_players)
        self.round_number = 0

    def _create_players(self, num_players: int, num_automated_players: int) -> List[Player]:
        players = []
        for i in range(num_players):
            if i < num_automated_players:
                if i == 0:  # First automated player uses card counting
                    players.append(CardCountingPlayer(f"Card Counter {i+1}", is_automated=True, chips=1000, threshold=-2))
                else:
                    players.append(Player(f"Player {i+1}", is_automated=True, chips=1000))
            else:
                players.append(Player("User", is_automated=False, chips=1000))
        return players

    def play_round(self):
        self.round_number += 1
        print(f"\n--- Round {self.round_number} ---")

        user = next((player for player in self.players if player.name == "User"), None)
        if user and user.chips <= 0:
            print(f"{user.name} has run out of chips. Game over!")
            return False

        for player in self.players:
            player.place_bet()

        for _ in range(2):
            for player in self.players:
                player.hand.append(self.deck.deal_card())
            self.dealer.hand.append(self.deck.deal_card())

        print(f"\nDealer's Hand: [{self.dealer.hand[0]}, ?]")
        for player in self.players:
            print(f"{player.name}'s Hand: {player.hand} (Value: {player.hand_value()})")

        for player in self.players:
            while player.hand_value() < 21:
                action = player.decide_action(self.dealer.hand[0])
                if action == 'hit':
                    player.hand.append(self.deck.deal_card())
                    print(f"{player.name} hits. Hand: {player.hand} (Value: {player.hand_value()})")
                else:
                    print(f"{player.name} stands.")
                    break

        print(f"\nDealer's Hand: {self.dealer.hand} (Value: {self.dealer.hand_value()})")
        while self.dealer.decide_action() == 'hit':
            self.dealer.hand.append(self.deck.deal_card())
            print(f"Dealer hits. Hand: {self.dealer.hand} (Value: {self.dealer.hand_value()})")

        # Determine outcomes
        dealer_value = self.dealer.hand_value()
        for player in self.players:
            player_value = player.hand_value()
            if player_value > 21:
                print(f"{player.name} busts! Loses {player.bet} chips.")
            elif dealer_value > 21 or player_value > dealer_value:
                player.chips += 2 * player.bet
                print(f"{player.name} wins! Wins {player.bet} chips.")
            elif player_value == dealer_value:
                player.chips += player.bet
                print(f"{player.name} pushes. Gets back {player.bet} chips.")
            else:
                print(f"{player.name} loses! Loses {player.bet} chips.")

        for player in self.players:
            player.reset_hand()
        self.dealer.reset_hand()

        return True

    def reset_game(self):
        self.deck.reset()
        for player in self.players:
            player.reset_hand()
        self.dealer.reset_hand()

class Simulation:
    def __init__(self, num_games: int, num_players: int, num_automated_players: int, num_decks: int):
        self.num_games = num_games
        self.num_players = num_players
        self.num_automated_players = num_automated_players
        self.num_decks = num_decks
        self.results = []

    def run_simulation(self):
        game = BlackjackGame(self.num_players, self.num_automated_players, self.num_decks)
        for _ in range(self.num_games):
            if not game.play_round():
                break
            self.results.append([player.chips for player in game.players])
            game.reset_game()

    def analyze_results(self):
        for i, player_results in enumerate(zip(*self.results)):
            avg_chips = sum(player_results) / len(player_results)
            if i < self.num_automated_players:
                if i == 0:  # First automated player is the card counter
                    print(f"Card Counter {i+1} average chips per game: {avg_chips:.2f}")
                else:
                    print(f"Player {i+1} average chips per game: {avg_chips:.2f}")
            else:
                print(f"User average chips per game: {avg_chips:.2f}")

if __name__ == "__main__":
    simulation = Simulation(num_games=3, num_players=4, num_automated_players=3, num_decks=6)
    simulation.run_simulation()
    simulation.analyze_results()

7. Create a test scenario where one player, using the above strategy, is playing with a dealer and 3 other players that follow the dealer's strategy. Each player starts with same number of chips. Play 50 rounds (or until the strategy player is out of money). Compute the strategy player's winnings. You may remove unnecessary printouts from your code (perhaps implement a verbose/quiet mode) to reduce the output.

In [ ]:
import random
from typing import List

class Card:
    def __init__(self, suit: str, rank: str):
        self.suit = suit
        self.rank = rank

    def value(self) -> int:
        if self.rank in ['J', 'Q', 'K']:
            return 10
        elif self.rank == 'A':
            return 11
        else:
            return int(self.rank)

    def __repr__(self):
        return f"{self.rank} of {self.suit}"

class Deck:
    def __init__(self, num_decks: int):
        self.cards = []
        self.num_decks = num_decks
        self.reset()

    def reset(self):
        suits = ['Spade', 'Diamond', 'Club', 'Heart']
        ranks = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A']
        self.cards = [Card(suit, rank) for suit in suits for rank in ranks] * self.num_decks
        self.shuffle()

    def shuffle(self):
        random.shuffle(self.cards)

    def deal_card(self) -> Card:
        return self.cards.pop()

class Player:
    def __init__(self, name: str, is_automated: bool, chips: int):
        self.name = name
        self.is_automated = is_automated
        self.chips = chips
        self.hand = []
        self.bet = 0

    def place_bet(self) -> int:
        if self.is_automated:
            self.bet = random.randint(1, min(100, self.chips))
            self.chips -= self.bet
            return self.bet
        else:
            self.bet = 10 
            self.chips -= self.bet
            return self.bet

    def decide_action(self, dealer_up_card: Card) -> str:
        if self.is_automated:
            # Basic strategy: hit if hand value < 16, stand otherwise
            if self.hand_value() < 16:
                return 'hit'
            else:
                return 'stand'
        else:
            return 'stand' 

    def hand_value(self) -> int:
        value = sum(card.value() for card in self.hand)
        aces = sum(1 for card in self.hand if card.rank == 'A')
        while value > 21 and aces:
            value -= 10
            aces -= 1
        return value

    def reset_hand(self):
        self.hand = []
        self.bet = 0

    def __repr__(self):
        return f"{self.name} (Chips: {self.chips})"

class CardCountingPlayer(Player):
    def __init__(self, name: str, is_automated: bool, chips: int, threshold: int = 0):
        super().__init__(name, is_automated, chips)
        self.running_count = 0  
        self.threshold = threshold  

    def update_running_count(self, card: Card):
        if 2 <= card.value() <= 6:
            self.running_count += 1
        elif 7 <= card.value() <= 9:
            self.running_count += 0
        elif 10 <= card.value() <= 11:
            self.running_count -= 1

    def decide_action(self, dealer_up_card: Card) -> str:
        if self.running_count < self.threshold:
            return 'hit'
        else:
            return 'stand'

    def reset_hand(self):
        for card in self.hand:
            self.update_running_count(card)
        super().reset_hand()

class Dealer:
    def __init__(self):
        self.hand = []

    def decide_action(self) -> str:
        if self.hand_value() < 16:
            return 'hit'
        else:
            return 'stand'

    def hand_value(self) -> int:
        value = sum(card.value() for card in self.hand)
        aces = sum(1 for card in self.hand if card.rank == 'A')
        while value > 21 and aces:
            value -= 10
            aces -= 1
        return value

    def reset_hand(self):
        self.hand = []

    def __repr__(self):
        return "Dealer"

class BlackjackGame:
    def __init__(self, num_players: int, num_automated_players: int, num_decks: int):
        self.deck = Deck(num_decks)
        self.dealer = Dealer()
        self.players = self._create_players(num_players, num_automated_players)
        self.round_number = 0

    def _create_players(self, num_players: int, num_automated_players: int) -> List[Player]:
        players = []
        for i in range(num_players):
            if i == 0:  # First player uses card counting
                players.append(CardCountingPlayer(f"Card Counter", is_automated=True, chips=1000, threshold=-2))
            else:
                players.append(Player(f"Player {i}", is_automated=True, chips=1000))
        return players

    def play_round(self):
        self.round_number += 1

        card_counter = next((player for player in self.players if isinstance(player, CardCountingPlayer)), None)
        if card_counter and card_counter.chips <= 0:
            return False

        for player in self.players:
            player.place_bet()

        for _ in range(2):
            for player in self.players:
                player.hand.append(self.deck.deal_card())
            self.dealer.hand.append(self.deck.deal_card())

        for player in self.players:
            while player.hand_value() < 21:
                action = player.decide_action(self.dealer.hand[0])
                if action == 'hit':
                    player.hand.append(self.deck.deal_card())

        while self.dealer.decide_action() == 'hit':
            self.dealer.hand.append(self.deck.deal_card())

        dealer_value = self.dealer.hand_value()
        for player in self.players:
            player_value = player.hand_value()
            if player_value > 21:
                pass  # Player busts
            elif dealer_value > 21 or player_value > dealer_value:
                player.chips += 2 * player.bet
            elif player_value == dealer_value:
                player.chips += player.bet
            else:
                pass  # Player loses

        for player in self.players:
            player.reset_hand()
        self.dealer.reset_hand()

        return True

    def reset_game(self):
        self.deck.reset()
        for player in self.players:
            player.reset_hand()
        self.dealer.reset_hand()

# Test scenario
def test():
    num_players = 4  # 1 card counter + 3 basic players
    num_automated_players = 4  
    num_decks = 6
    num_rounds = 50
    num_games = 50

    total_winnings = 0

    for game_num in range(num_games):
        game = BlackjackGame(num_players, num_automated_players, num_decks)
        card_counter = next((player for player in game.players if isinstance(player, CardCountingPlayer)), None)

        for round in range(num_rounds):
            if not game.play_round():
                break

        if card_counter:
            total_winnings += card_counter.chips - 1000

    print(f"Strategy player's total winnings over {num_games} games: {total_winnings}")

test()

8. Create a loop that runs 100 games of 50 rounds, as setup in previous question, and store the strategy player's chips at the end of the game (aka "winnings") in a list. Histogram the winnings. What is the average winnings per round? What is the standard deviation. What is the probabilty of net winning or lossing after 50 rounds?


In [ ]:
import random
from typing import List
import matplotlib.pyplot as plt
import numpy as np

class Card:
    def __init__(self, suit: str, rank: str):
        self.suit = suit
        self.rank = rank

    def value(self) -> int:
        if self.rank in ['J', 'Q', 'K']:
            return 10
        elif self.rank == 'A':
            return 11
        else:
            return int(self.rank)

    def __repr__(self):
        return f"{self.rank} of {self.suit}"

class Deck:
    def __init__(self, num_decks: int):
        self.cards = []
        self.num_decks = num_decks
        self.reset()

    def reset(self):
        suits = ['Spade', 'Diamond', 'Club', 'Heart']
        ranks = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A']
        self.cards = [Card(suit, rank) for suit in suits for rank in ranks] * self.num_decks
        self.shuffle()

    def shuffle(self):
        random.shuffle(self.cards)

    def deal_card(self) -> Card:
        return self.cards.pop()

class Player:
    def __init__(self, name: str, is_automated: bool, chips: int):
        self.name = name
        self.is_automated = is_automated
        self.chips = chips
        self.hand = []
        self.bet = 0

    def place_bet(self) -> int:
        if self.is_automated:
            self.bet = random.randint(1, min(100, self.chips))
            self.chips -= self.bet
            return self.bet
        else:
            self.bet = 10  
            self.chips -= self.bet
            return self.bet

    def decide_action(self, dealer_up_card: Card) -> str:
        if self.is_automated:
            # Basic strategy: hit if hand value < 16, stand otherwise
            if self.hand_value() < 16:
                return 'hit'
            else:
                return 'stand'
        else:
            return 'stand'  

    def hand_value(self) -> int:
        value = sum(card.value() for card in self.hand)
        aces = sum(1 for card in self.hand if card.rank == 'A')
        while value > 21 and aces:
            value -= 10
            aces -= 1
        return value

    def reset_hand(self):
        self.hand = []
        self.bet = 0

    def __repr__(self):
        return f"{self.name} (Chips: {self.chips})"

class CardCountingPlayer(Player):
    def __init__(self, name: str, is_automated: bool, chips: int, threshold: int = 0):
        super().__init__(name, is_automated, chips)
        self.running_count = 0 
        self.threshold = threshold  # Threshold for hit/stay decision

    def update_running_count(self, card: Card):
        if 2 <= card.value() <= 6:
            self.running_count += 1
        elif 7 <= card.value() <= 9:
            self.running_count += 0
        elif 10 <= card.value() <= 11:
            self.running_count -= 1

    def decide_action(self, dealer_up_card: Card) -> str:
        if self.running_count < self.threshold:
            return 'hit'
        else:
            return 'stand'

    def reset_hand(self):
        for card in self.hand:
            self.update_running_count(card)
        super().reset_hand()

class Dealer:
    def __init__(self):
        self.hand = []

    def decide_action(self) -> str:
        if self.hand_value() < 16:
            return 'hit'
        else:
            return 'stand'

    def hand_value(self) -> int:
        value = sum(card.value() for card in self.hand)
        aces = sum(1 for card in self.hand if card.rank == 'A')
        while value > 21 and aces:
            value -= 10
            aces -= 1
        return value

    def reset_hand(self):
        self.hand = []

    def __repr__(self):
        return "Dealer"

class BlackjackGame:
    def __init__(self, num_players: int, num_automated_players: int, num_decks: int):
        self.deck = Deck(num_decks)
        self.dealer = Dealer()
        self.players = self._create_players(num_players, num_automated_players)
        self.round_number = 0

    def _create_players(self, num_players: int, num_automated_players: int) -> List[Player]:
        players = []
        for i in range(num_players):
            if i == 0:  # First player uses card counting
                players.append(CardCountingPlayer(f"Card Counter", is_automated=True, chips=1000, threshold=-2))
            else:
                players.append(Player(f"Player {i}", is_automated=True, chips=1000))
        return players

    def play_round(self):
        self.round_number += 1

        card_counter = next((player for player in self.players if isinstance(player, CardCountingPlayer)), None)
        if card_counter and card_counter.chips <= 0:
            return False

        for player in self.players:
            player.place_bet()

        for _ in range(2):
            for player in self.players:
                player.hand.append(self.deck.deal_card())
            self.dealer.hand.append(self.deck.deal_card())

        for player in self.players:
            while player.hand_value() < 21:
                action = player.decide_action(self.dealer.hand[0])
                if action == 'hit':
                    player.hand.append(self.deck.deal_card())

        while self.dealer.decide_action() == 'hit':
            self.dealer.hand.append(self.deck.deal_card())

        dealer_value = self.dealer.hand_value()
        for player in self.players:
            player_value = player.hand_value()
            if player_value > 21:
                pass  
            elif dealer_value > 21 or player_value > dealer_value:
                player.chips += 2 * player.bet
            elif player_value == dealer_value:
                player.chips += player.bet
            else:
                pass  # Player loses

        for player in self.players:
            player.reset_hand()
        self.dealer.reset_hand()

        return True

    def reset_game(self):
        self.deck.reset()
        for player in self.players:
            player.reset_hand()
        self.dealer.reset_hand()

def test():
    num_players = 4  # 1 card counter + 3 basic players
    num_automated_players = 4  
    num_decks = 6
    num_rounds = 50
    num_games = 100

    winnings_list = []

    for game_num in range(num_games):
        game = BlackjackGame(num_players, num_automated_players, num_decks)
        card_counter = next((player for player in game.players if isinstance(player, CardCountingPlayer)), None)

        for round in range(num_rounds):
            if not game.play_round():
                break

        if card_counter:
            winnings_list.append(card_counter.chips - 1000)

    plt.hist(winnings_list, bins=20, edgecolor='black')
    plt.title("Histogram of Strategy Player's Winnings")
    plt.xlabel("Winnings")
    plt.ylabel("Frequency")
    plt.show()

    winnings_array = np.array(winnings_list)
    average_winnings = np.mean(winnings_array)
    std_dev_winnings = np.std(winnings_array)
    prob_win = np.mean(winnings_array > 0)
    prob_loss = np.mean(winnings_array < 0)

    print(f"Average winnings per game: {average_winnings:.2f}")
    print(f"Standard deviation of winnings: {std_dev_winnings:.2f}")
    print(f"Probability of net winning after 50 rounds: {prob_win:.2f}")
    print(f"Probability of net losing after 50 rounds: {prob_loss:.2f}")

test()

9. Repeat previous questions scanning the value of the threshold. Try at least 5 different threshold values. Can you find an optimal value?

10. Create a new strategy based on web searches or your own ideas. Demonstrate that the new strategy will result in increased or decreased winnings. 